# Creation full features

Aim is to conciliate our training test with the external data

## Loading the data

In [1]:
import pandas as pd

# Let's load our data

rough_external = pd.read_csv('./data/external_data.csv')
rough_trainig = pd.read_parquet('train.parquet')

Let's define the functions we chose in 'data_exploration_adrien' and 'external_data_explo_adrien' to process the features

In [2]:
# Let's begin by the function to process the training data

# A function to get the dates from the 'date' column and put it into index
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # Let's put the date into index
    X = X.set_index(X["date"])
    X = X.sort_index()

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])


# delete the unuseful columns
def _delete_columns(X):
    X = X.copy()  # modify a copy of X
    col_delete = [
        "bike_count",  # I only keep the log_bike_count # à mettre et enlever selon si test set ou trianing
        "counter_id",  # I only keep the site_id
        "counter_name",  # same
        "site_name",  # same
        "counter_technical_id",  # same
        "coordinates",  # I prefer to get latitude and longitude
        "counter_installation_date",
    ]  # for my example I remove it because can't fit the model with that but still a data to use
    X = X.drop(columns=col_delete)
    return X


# A function that aggregates all the encoding
def _process_features(X):
    X = X.copy()  # modify a copy of X
    X = _encode_dates(X)
    X = _delete_columns(X)
    return X

In [3]:
# Let's begin by the function to process the test data

# A function to get the dates from the 'date' column and put it into index
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # Let's put the date into index
    X = X.set_index(X["date"])
    X = X.sort_index()

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])


# delete the unuseful columns
def _delete_columns_test(X):
    X = X.copy()  # modify a copy of X
    col_delete = [
        "counter_id",  # I only keep the site_id
        "counter_name",  # same
        "site_name",  # same
        "counter_technical_id",  # same
        "coordinates",  # I prefer to get latitude and longitude
        "counter_installation_date",
    ]  # for my example I remove it because can't fit the model with that but still a data to use
    X = X.drop(columns=col_delete)
    return X


# A function that aggregates all the encoding
def _process_features_test(X):
    X = X.copy()  # modify a copy of X
    X = _encode_dates(X)
    X = _delete_columns_test(X)
    return X

In [4]:
# Let's continue with the function to process the external data

# Function to remove all the columns that have more than 45% of NaN values
def _delete_na_columns(X):
    X = X.copy()
    mute_columns = []
    for elem in X.columns:
        if X[elem].isna().sum() / X[elem].shape[0] > 0.40:
            mute_columns.append(elem)
    X = X.drop(columns=mute_columns)
    return X


# On va mettre la date en index et classer le tout par ordre chronologique

def _datetime_ordered_index(X):
    X = X.copy()
    X = X.set_index(X['date'])
    X = X.sort_index()
    X = X.drop(columns=["date"])
    return X



# Now we want to replace the null values by the last non missing value
def _fill_na_with_ffill_or_mode(X):
    X = X.copy()
    X = X.fillna(method='ffill') # replacing NaN by the last non missing value
    
    # For the few values that couldn't be filled = most frequent value
    for col in X.columns:
        if X[col].isna().any():  
            mode_value = X[col].mode().iloc[0]
            X[col].fillna(mode_value, inplace=True)
    
    return X


# Let's choose the features that we think are the most relevant in our case
def _choose_external_features(X):
    col = ['t', # temperature, obvious factor
           'ww', # description of the weather situation
           'cl', # cloud covering
           'tend24', # weather tendency on 24h
           'ff', # wind
           'etat_sol', # state of the floor
           'rr3'] # rain during last 3 hours
    X = X.copy()
    X = X[col]
    return X


def _process_external_features(X):
    X = X.copy()
    X = _delete_na_columns(X)
    X = _datetime_ordered_index(X)
    X = _fill_na_with_ffill_or_mode(X)
    X = _choose_external_features(X)
    return X


In [5]:
# Let's get clean datasets

training_data = _process_features(rough_trainig)
external_data = _process_external_features(rough_external)

In [6]:
training_data.head()

,site_id,latitude,longitude,log_bike_count,year,month,day,weekday,hour
date,,,,,,,,,
2020-09-01 01:00:00,100007049,48.846028,2.375429,0.693147,2020,9,1,1,1
2020-09-01 01:00:00,100056226,48.830331,2.400551,0.693147,2020,9,1,1,1
2020-09-01 01:00:00,100047545,48.890457,2.368852,0.693147,2020,9,1,1,1
2020-09-01 01:00:00,100060178,48.846380,2.315290,3.091042,2020,9,1,1,1
2020-09-01 01:00:00,100056327,48.864610,2.409690,1.098612,2020,9,1,1,1


In [7]:
external_data.head()

,t,ww,cl,tend24,ff,etat_sol,rr3
date,,,,,,,
2020-09-01 00:00:00,285.75,1,30.0,120.0,1.6,0.0,0.0
2020-09-01 03:00:00,283.95,2,30.0,0.0,1.1,0.0,0.0
2020-09-01 06:00:00,284.25,3,30.0,-60.0,1.8,0.0,0.0
2020-09-01 09:00:00,291.25,1,31.0,-130.0,2.9,0.0,0.0
2020-09-01 12:00:00,293.95,0,31.0,-230.0,2.6,0.0,0.0


In [8]:
training_data.shape

(496827, 9)

In [9]:
training_data_initial = training_data
external_data_initial = external_data
training_data = training_data_initial.copy()
external_data = external_data_initial.copy()

## Conciliate the training data with the external data

In [10]:
training_data.index = pd.to_datetime(training_data.index)
external_data.index = pd.to_datetime(external_data.index)


In [11]:
training_data["date"] = training_data.index
external_data["date"] = external_data.index


In [12]:
# Supprimer la colonne 'date' si elle existe déjà
## ça m'est arrivé pcq j'avais fait un truc pourri avant mais a priori pas besoin
if "date" in training_data.columns:
    training_data = training_data.drop(columns=["date"])

if "date" in external_data.columns:
    external_data = external_data.drop(columns=["date"])

# Réinitialiser l'index et renommer la colonne "index" en "date"
training_data = training_data.reset_index().rename(columns={"index": "date"})
external_data = external_data.reset_index().rename(columns={"index": "date"})

# Trier external_data par date
external_data = external_data.sort_values(by="date")

# Fonction pour trouver la date la plus proche
def find_closest_date_binary(row, external_data):
    idx = external_data["date"].searchsorted(row["date"], side="left")
    if idx == 0:
        closest_row = external_data.iloc[0]
    elif idx == len(external_data):
        closest_row = external_data.iloc[-1]
    else:
        before = external_data.iloc[idx - 1]
        after = external_data.iloc[idx]
        closest_row = before if abs(before["date"] - row["date"]) < abs(after["date"] - row["date"]) else after
    return closest_row

# Appliquer la recherche optimisée
external_cols = external_data.columns.drop("date")  # Exclure la colonne "date"
for col in external_cols:
    training_data[col] = training_data.apply(
        lambda row: find_closest_date_binary(row, external_data)[col], axis=1
    )


In [13]:
training_data.isna().sum()

date              0
site_id           0
latitude          0
longitude         0
log_bike_count    0
year              0
month             0
day               0
weekday           0
hour              0
t                 0
ww                0
cl                0
tend24            0
ff                0
etat_sol          0
rr3               0
dtype: int64

In [14]:
training_data = training_data.drop(columns='date')

## Submission test with new features

### Pipeline and fit

In [15]:
X = training_data.drop(columns='log_bike_count')
y = training_data['log_bike_count']

In [16]:
training_data.head()

,site_id,latitude,longitude,log_bike_count,year,month,day,weekday,hour,t,ww,cl,tend24,ff,etat_sol,rr3
0,100007049,48.846028,2.375429,0.693147,2020,9,1,1,1,285.75,1,30.0,120.0,1.6,0.0,0.0
1,100056226,48.830331,2.400551,0.693147,2020,9,1,1,1,285.75,1,30.0,120.0,1.6,0.0,0.0
2,100047545,48.890457,2.368852,0.693147,2020,9,1,1,1,285.75,1,30.0,120.0,1.6,0.0,0.0
3,100060178,48.846380,2.315290,3.091042,2020,9,1,1,1,285.75,1,30.0,120.0,1.6,0.0,0.0
4,100056327,48.864610,2.409690,1.098612,2020,9,1,1,1,285.75,1,30.0,120.0,1.6,0.0,0.0


In [17]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
   #('scaler', StandardScaler()),  # la normalisation des données n'a quasiment pas d'impact ici donc je la mets pas
    ('regressor', GradientBoostingRegressor())  # on utilise ici un modèle assez simple, possible de changer facilement
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# cross validation
cv_scores = cross_val_score(pipeline, X, y, cv=4, scoring='neg_mean_squared_error')
print(f"Cross-validated Mean Squared Error: {-cv_scores.mean()}")

Mean Squared Error: 0.5842669937872806
Cross-validated Mean Squared Error: 0.7510017475729407


### Prediction

In [18]:
test_initial = pd.read_parquet('final_test.parquet')
test_data = _process_features_test(test_initial)
external_data = _process_external_features(rough_external)


In [19]:
test_data.index = pd.to_datetime(test_data.index)
external_data.index = pd.to_datetime(external_data.index)


In [20]:
test_data["date"] = test_data.index
external_data["date"] = external_data.index


In [21]:
external_data.head()

,t,ww,cl,tend24,ff,etat_sol,rr3,date
date,,,,,,,,
2020-09-01 00:00:00,285.75,1,30.0,120.0,1.6,0.0,0.0,2020-09-01 00:00:00
2020-09-01 03:00:00,283.95,2,30.0,0.0,1.1,0.0,0.0,2020-09-01 03:00:00
2020-09-01 06:00:00,284.25,3,30.0,-60.0,1.8,0.0,0.0,2020-09-01 06:00:00
2020-09-01 09:00:00,291.25,1,31.0,-130.0,2.9,0.0,0.0,2020-09-01 09:00:00
2020-09-01 12:00:00,293.95,0,31.0,-230.0,2.6,0.0,0.0,2020-09-01 12:00:00


In [22]:
test_data.head()

,site_id,latitude,longitude,year,month,day,weekday,hour,date
date,,,,,,,,,
2021-09-10 01:00:00,100007049,48.846028,2.375429,2021,9,10,4,1,2021-09-10 01:00:00
2021-09-10 01:00:00,100036718,48.834360,2.377000,2021,9,10,4,1,2021-09-10 01:00:00
2021-09-10 01:00:00,100057380,48.864620,2.314440,2021,9,10,4,1,2021-09-10 01:00:00
2021-09-10 01:00:00,100056223,48.862840,2.310345,2021,9,10,4,1,2021-09-10 01:00:00
2021-09-10 01:00:00,100056226,48.830331,2.400551,2021,9,10,4,1,2021-09-10 01:00:00


In [23]:
test_data.columns

Index(['site_id', 'latitude', 'longitude', 'year', 'month', 'day', 'weekday',
       'hour', 'date'],
      dtype='object')

In [24]:
# Supprimer la colonne 'date' si elle existe déjà
## ça m'est arrivé pcq j'avais fait un truc pourri avant mais a priori pas besoin
if "date" in test_data.columns:
  test_data = test_data.drop(columns=["date"])

if "date" in external_data.columns:
    external_data = external_data.drop(columns=["date"])

# Réinitialiser l'index et renommer la colonne "index" en "date"
test_data = test_data.reset_index().rename(columns={"index": "date"})
external_data = external_data.reset_index().rename(columns={"index": "date"})

# Trier external_data par date
external_data = external_data.sort_values(by="date")

# Fonction pour trouver la date la plus proche
def find_closest_date_binary(row, external_data):
    idx = external_data["date"].searchsorted(row["date"], side="left")
    if idx == 0:
        closest_row = external_data.iloc[0]
    elif idx == len(external_data):
        closest_row = external_data.iloc[-1]
    else:
        before = external_data.iloc[idx - 1]
        after = external_data.iloc[idx]
        closest_row = before if abs(before["date"] - row["date"]) < abs(after["date"] - row["date"]) else after
    return closest_row

# Appliquer la recherche optimisée
external_cols = external_data.columns.drop("date")  # Exclure la colonne "date"
for col in external_cols:
    test_data[col] = test_data.apply(
        lambda row: find_closest_date_binary(row, external_data)[col], axis=1
    )


In [25]:
test_final = test_data.copy()
test_final = test_final.drop(columns='date')

In [26]:
import numpy as np

y_pred = pipeline.predict(test_final)
results = pd.DataFrame(
    dict(
        Id=np.arange(y_pred.shape[0]),
        log_bike_count=y_pred,
    )
)
results.to_csv("submission_full_features.csv", index=False)

In [27]:
# LA submission full feature ici donne une RMSE de 2.03, ce qui ne correspond pas du tout à ce que dit la cross val, 
# il y a probablement un problème mais faut voir où il est